In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
ls /kaggle/input/train-test-paths

train_test_paths/


In [24]:
import shutil

def delete_extra_folders():
    for root, dirname, filenames in os.walk('/kaggle/input'):
        for dir in dirname:
            if (dir=='extra' or dir=='Extra'):
                file_path = os.path.join(root,dir)
                if os.path.exists(file_path):
                    print(file_path)
                    shutil.rmtree(file_path)  
#delete_extra_folders()

In [25]:
#delete_extra_folders()

In [26]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [27]:
import mediapipe as mp

In [28]:
ls /kaggle/input/include-50/

Adjectives/  Clothes/        Electronics/  Home/  Means_of_Transportation/
Animals/     Days_and_Time/  Greetings/    Jobs/


In [29]:
ls

__notebook_source__.ipynb  keypoints1/


In [30]:
mkdir keypoints1

mkdir: cannot create directory ‘keypoints1’: File exists


In [57]:
import os
import json
import multiprocessing
import argparse
import os.path
import cv2
import mediapipe as mp
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import numpy as np
import gc
import warnings

def process_landmarks(landmarks):
    x_list, y_list = [], []
    if landmarks is not None:
        for landmark in landmarks.landmark:
            x_list.append(landmark.x)
            y_list.append(landmark.y)
    return x_list, y_list


def process_hand_keypoints(results):
    hand1_x, hand1_y, hand2_x, hand2_y = [], [], [], []

    if results.multi_hand_landmarks is not None:
        if len(results.multi_hand_landmarks) > 0:
            hand1 = results.multi_hand_landmarks[0]
            hand1_x, hand1_y = process_landmarks(hand1)

        if len(results.multi_hand_landmarks) > 1:
            hand2 = results.multi_hand_landmarks[1]
            hand2_x, hand2_y = process_landmarks(hand2)

    return hand1_x, hand1_y, hand2_x, hand2_y


def process_pose_keypoints(results):
    pose = results.pose_landmarks
    pose_x, pose_y = process_landmarks(pose)
    return pose_x, pose_y


def swap_hands(pose_x, pose_y, hand, input_hand):
    left_wrist_x, left_wrist_y = np.nan, np.nan
    right_wrist_x, right_wrist_y = np.nan, np.nan

    if len(pose_x) >= 17 and len(pose_y) >= 17:
        left_wrist_x, left_wrist_y = pose_x[15], pose_y[15]
        right_wrist_x, right_wrist_y = pose_x[16], pose_y[16]
        
    hand_x, hand_y = hand

    left_dist = (left_wrist_x - hand_x) ** 2 + (left_wrist_y - hand_y) ** 2
    right_dist = (right_wrist_x - hand_x) ** 2 + (right_wrist_y - hand_y) ** 2

    if left_dist < right_dist and input_hand == "h2":
        return True

    if right_dist < left_dist and input_hand == "h1":
        return True

    return False


def process_video(path, save_dir):
    hands = mp.solutions.hands.Hands(
        min_detection_confidence=0.5, min_tracking_confidence=0.5
    )
    pose = mp.solutions.pose.Pose(
        min_detection_confidence=0.5, min_tracking_confidence=0.5#, upper_body_only=True
    )

    pose_points_x, pose_points_y = [], []
    hand1_points_x, hand1_points_y = [], []
    hand2_points_x, hand2_points_y = [], []
    
    
    # skip processing if 'extra' or 'Extra' is present in path
#     if 'extra' in path or 'Extra' in path:
#         print(f"Skipping {path}")
#         return
    print(path)
    label = path.split("/")[-2]
    label = "".join([i for i in label if i.isalpha()]).lower()
    uid = os.path.splitext(os.path.basename(path))[0]
    uid = "_".join([label, uid])
    n_frames = 0
    if not os.path.isfile(path):
        warnings.warn(path + " file not found")
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        ret, image = cap.read()
        if not ret:
            break
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        hand_results = hands.process(image)
        pose_results = pose.process(image)

        hand1_x, hand1_y, hand2_x, hand2_y = process_hand_keypoints(hand_results)
        pose_x, pose_y = process_pose_keypoints(pose_results)
        
        ## Assign hands to correct positions
        if len(hand1_x) > 0 and len(hand2_x) == 0:
            if swap_hands(
                 pose_x,
                 pose_y,
                 hand=(hand1_x[0], hand1_y[0]),
                 input_hand="h1",
                ):
                hand1_x, hand1_y, hand2_x, hand2_y = hand2_x, hand2_y, hand1_x, hand1_y

        elif len(hand1_x) == 0 and len(hand2_x) > 0:
            if swap_hands(
                 pose_x,
                 pose_y,
                 hand=(hand2_x[0], hand2_y[0]),
                 input_hand="h2",
             ):
                hand1_x, hand1_y, hand2_x, hand2_y = hand2_x, hand2_y, hand1_x, hand1_y

         ## Set to nan so that values can be interpolated in dataloader
        pose_x = pose_x if pose_x else [np.nan] * 25
        pose_y = pose_y if pose_y else [np.nan] * 25

        hand1_x = hand1_x if hand1_x else [np.nan] * 21
        hand1_y = hand1_y if hand1_y else [np.nan] * 21
        hand2_x = hand2_x if hand2_x else [np.nan] * 21
        hand2_y = hand2_y if hand2_y else [np.nan] * 21

        pose_points_x.append(pose_x)
        pose_points_y.append(pose_y)
        hand1_points_x.append(hand1_x)
        hand1_points_y.append(hand1_y)
        hand2_points_x.append(hand2_x)
        hand2_points_y.append(hand2_y)

        n_frames += 1

    cap.release()

    ## Set to nan so that values can be interpolated in dataloader
    pose_points_x = pose_points_x if pose_points_x else [[np.nan] * 25]
    pose_points_y = pose_points_y if pose_points_y else [[np.nan] * 25]

    hand1_points_x = hand1_points_x if hand1_points_x else [[np.nan] * 21]
    hand1_points_y = hand1_points_y if hand1_points_y else [[np.nan] * 21]
    hand2_points_x = hand2_points_x if hand2_points_x else [[np.nan] * 21]
    hand2_points_y = hand2_points_y if hand2_points_y else [[np.nan] * 21]

    save_data = {
        "uid": uid,
        "label": label,
        "pose_x": pose_points_x,
        "pose_y": pose_points_y,
        "hand1_x": hand1_points_x,
        "hand1_y": hand1_points_y,
        "hand2_x": hand2_points_x,
        "hand2_y": hand2_points_y,
        "n_frames": n_frames,
    }
    with open(os.path.join(save_dir, f"{uid}.json"), "w") as f:
        json.dump(save_data, f)

    hands.close()
    pose.close()
    del hands, pose, save_data
    gc.collect()

def correct_paths(paths):
    """
    If the video is not in include 50m then it can be in include-50-2, so update path to check there.
    And also check if MOV or MP4 exension works
    """
    n = len(paths)
    for i in range(n):
        if not os.path.exists(paths[i]):
            new_path = paths[i].replace('MOV','MP4')
            if(os.path.exists(new_path)):
                paths[i] = new_path
            else:
                paths[i] = paths[i].replace('include-50','include-50-2')
                if(not os.path.exists(paths[i])):
                    paths[i] = paths[i].replace('MOV','MP4')
               
    return paths

def load_file(path, include_dir):
    with open(path, "r") as fp:
        data = fp.read()
        data = data.split("\n")
    data = list(map(lambda x: os.path.join(include_dir, x), data))
    data = correct_paths(data)
    return data


def load_train_test_val_paths(args):
    train_paths = load_file(
        f"/kaggle/input/train-test-paths/train_test_paths/{args['dataset']}_train.txt", args['include_dir']
    )
    val_paths = load_file(f"/kaggle/input/train-test-paths/train_test_paths/{args['dataset']}_val.txt", args['include_dir'])
    test_paths = load_file(
        f"/kaggle/input/train-test-paths/train_test_paths/{args['dataset']}_test.txt", args['include_dir']
    )
    return train_paths, val_paths, test_paths

def save_keypoints(dataset, file_paths, mode, args):
    save_dir = os.path.join(args['save_direc'], f"{dataset}_keypoints")
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    Parallel(n_jobs=n_cores, backend="multiprocessing")(
        delayed(process_video)(path, save_dir)
        for path in tqdm(file_paths, desc=f"processing videos")
    )
#689 train
#77 val
#192 test

args = {
    'include_dir':'/kaggle/input/include-50/',
    #/kaggle/input/include-50-2/
    'save_direc':'/kaggle/working/keypoints1/',
    'dataset':'include50'
}

n_cores = multiprocessing.cpu_count()
train_paths, val_paths, test_paths = load_train_test_val_paths(args)

save_keypoints('include50', train_paths, "train", args)
#save_keypoints('include50', '/kaggle/input/include-50-2/', "train",args)

save_keypoints('include50', val_paths, "val", args)
#save_keypoints('include50', '/kaggle/input/include-50-2/', "val",args)

save_keypoints('include50', test_paths, "test", args)
#save_keypoints('include50', '/kaggle/input/include-50-2/', "test",args)

processing videos:   0%|          | 0/689 [00:00<?, ?it/s]

/kaggle/input/include-50/Animals/4. Bird/MVI_3040.MOV
/kaggle/input/include-50-2/Places/23. Court/MVI_3612.MOV/kaggle/input/include-50/Adjectives/94. good/MVI_9269.MOV
/kaggle/input/include-50/Greetings/48. Hello/MVI_0089.MOV



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/kaggle/input/include-50/Electronics/54. Cell phone/MVI_9100.MP4
/kaggle/input/include-50/Home/28. Window/MVI_4364.MOV
/kaggle/input/include-50/Adjectives/87. hot/MVI_5137.MOV
/kaggle/input/include-50/Days_and_Time/67. Monday/MVI_5009.MOV
/kaggle/input/include-50-2/Places/28. Store or Shop/MVI_3310.MOV
/kaggle/input/include-50/Means_of_Transportation/16. train ticket/MVI_3134.MOV
/kaggle/input/include-50-2/Places/35. Bank/MVI_3582.MOV
/kaggle/input/include-50/Greetings/48. Hello/MVI_9955.MOV
/kaggle/input/include-50-2/People/66. Brother/MVI_8631.MP4
/kaggle/input/include-50-2/People/77. Boy/MVI_5285.MOV
/kaggle/input/include-50/Adjectives/97. dry/MVI_9523.MOV
/kaggle/input/include-50/Greetings/48. Hello/MVI_9984.MOV
/kaggle/input/include-50/Jobs/84. Teacher/MVI_4744.MOV
/kaggle/input/include-50/Means_of_Transportation/11. Car/MVI_4177.MOV
/kaggle/input/include-50/Adjectives/3. happy/MVI_9455.MOV
/kaggle/input/include-50-2/Colours/47. Red/MVI_4018.MP4


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: UserWarning: /kaggle/input/include-50-2/Colours/47. Red/MVI_4018.MP4 file not found


...

/kaggle/input/include-50-2/Colours/47. Red/MVI_4898.MP4


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: UserWarning: /kaggle/input/include-50-2/Colours/47. Red/MVI_4898.MP4 file not found


/kaggle/input/include-50-2/Seasons/61. Summer/MVI_4991.MOV/kaggle/input/include-50-2/Seasons/61. Summer/MVI_9122.MP4

/kaggle/input/include-50-2/Seasons/61. Summer/MVI_4568.MOV
/kaggle/input/include-50-2/Colours/55. White/MVI_5211.MP4


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:90: UserWarning: /kaggle/input/include-50-2/Colours/55. White/MVI_5211.MP4 file not found


/kaggle/input/include-50/Adjectives/1. loud/MVI_9369.MOV
/kaggle/input/include-50/Adjectives/1. loud/MVI_9536.MOV
/kaggle/input/include-50/Adjectives/1. loud/MVI_5258.MOV
/kaggle/input/include-50/Adjectives/1. loud/MVI_5259.MOV
/kaggle/input/include-50/Adjectives/1. loud/MVI_9289.MOV
/kaggle/input/include-50/Adjectives/1. loud/MVI_5177.MOV
/kaggle/input/include-50/Adjectives/2. quiet/MVI_5180.MOV
/kaggle/input/include-50/Adjectives/2. quiet/MVI_9539.MOV
/kaggle/input/include-50/Adjectives/2. quiet/MVI_9372.MOV
/kaggle/input/include-50/Adjectives/3. happy/MVI_5183.MOV
/kaggle/input/include-50/Adjectives/3. happy/MVI_9297.MOV
/kaggle/input/include-50/Adjectives/3. happy/MVI_5263.MOV
/kaggle/input/include-50/Adjectives/78. long/MVI_9219.MOV
/kaggle/input/include-50/Adjectives/78. long/MVI_9379.MOV
/kaggle/input/include-50/Adjectives/78. long/MVI_5268.MOV
/kaggle/input/include-50/Adjectives/78. long/MVI_5106.MOV
/kaggle/input/include-50/Adjectives/78. long/MVI_9300.MOV
/kaggle/input/includ

In [37]:
count = 0
for path in train_paths:
    if not os.path.exists(path):
        count = count +1
count

299

In [ ]:
ls keypoints1/include50_keypoints/


To DO
1. Understand ai4bharat mediapipe code
2. debug it

done 3. use mediapipe from scratch for 1 vdo

In [ ]:
import time
for i in tqdm(range(10),desc=f"processing videos"):
    time.sleep(2)

In [ ]:
#ignore this box, this was me trying to understand opencv
cap = cv2.VideoCapture('/kaggle/input/include-50/Adjectives/1. loud/MVI_5177.MOV')
while cap.isOpened():
    ret, image = cap.read()
    if not ret:
        break
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


To DO for group:
1. make a dataframe, like previous one. hand1x,hand1y, hand2x, hand2y, label, n_frames.
2. instead of processing all frames, only select mid 30? or 40? and then process them.
3. write a function to extract keypoints, for that we have to understand the structure of hand_landmarker_result

In [ ]:
#our box

import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Create a hand landmarker instance with the video mode:
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='/kaggle/input/hand-landmark/hand_landmarker.task'),
    running_mode=VisionRunningMode.VIDEO)

with HandLandmarker.create_from_options(options) as landmarker:
    
  # The landmarker is initialized. Use it here.
  # ...
    # Use OpenCV’s VideoCapture to load the input video.
    cap = cv2.VideoCapture('/kaggle/input/include-50/Adjectives/1. loud/MVI_5177.MOV')
    video_framerate = cap.get(cv2.CAP_PROP_FPS)
    vdo_keypoints = []
    
    while cap.isOpened():
        ret, image = cap.read() #read returns if frame_exits, current_frame
        if not ret:
            break
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #when we load, its in blue green red, we convert it to RGB
    # Load the frame rate of the video using OpenCV’s CV_CAP_PROP_FPS
    # You’ll need it to calculate the timestamp for each frame.
    # Loop through each frame in the video using VideoCapture#read()
    # Convert the frame received from OpenCV to a MediaPipe’s Image object.
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
        # Perform hand landmarks detection on the provided single image.
        # The hand landmarker must be created with the video mode.
        frame_timestamp_ms = int(cap.get(cv2.CAP_PROP_POS_MSEC))
        print(type(frame_timestamp_ms))
        #case CV_FFMPEG_CAP_PROP_POS_MSEC:
        #return 1000.0*(double)frame_number/get_fps();
        hand_landmarker_result = landmarker.detect_for_video(mp_image, frame_timestamp_ms)
        print(hand_landmarker_result)
        hand1_x,hand1_y, hand2_x, hand2_y = process_hand_landmarker_result(hand_landmarker_result) 
        
        
        #process the output.
        #refer last section of this link
        #https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#video_2
        
        
        cap.release()
        
    
    

In [ ]:
print(type(hand_landmarker_result.hand_landmarks))

In [ ]:
print(type(hand_landmarker_result.handedness))

In [ ]:
print(type(hand_landmarker_result.hand_world_landmarks))

In [ ]:
print(len(hand_landmarker_result.hand_world_landmarks[0]))

In [ ]:
print(len(hand_landmarker_result.hand_landmarks[0]))

In [ ]:
hand_landmarker_result.hand_landmarks[0]

In [ ]:
hand_landmarker_result.hand_world_landmarks[0]